# Calculate feature vector from inception_v4 model using TF_exractor

In [1]:
import os
import h5py
import re
import pandas as pd

In [2]:
def setpath(path):
    os.chdir(path)
    print(os.getcwd())

In [3]:
def get_index(name):
    name = str(name)
    p_s = re.compile("[0-9]+.jpeg")
    p_e = re.compile(".jpeg")

    m_s = p_s.search(name)
    m_e = p_e.search(name)

    return name[m_s.start():m_e.start()]

In [4]:
username='haewonpark'
path_features = '/Users/'+username+'/Projects/handson/TF_FeatureExtraction/'
setpath(path_features)

/Users/haewonpark/Projects/handson/TF_FeatureExtraction


In [5]:
import sys
sys.path.append("/Users/"+username+"/Projects/handson/models/research/slim")
import nets

In [6]:
%run example_feat_extract.py --network inception_v4 --checkpoint ./checkpoints/inception_v4.ckpt --image_path ./images/ --out_file ./FV_inceptionV4.h5 --layer_names AuxLogits,global_pool,PreLogitsFlatten,Logits

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Restoring parameters from ./checkpoints/inception_v4.ckpt
Conv2d_1a_3x3 has shape (?, 149, 149, 32)
Conv2d_2a_3x3 has shape (?, 147, 147, 32)
Conv2d_2b_3x3 has shape (?, 147, 147, 64)
Mixed_3a has shape (?, 73, 73, 160)
Mixed_4a has shape (?, 71, 71, 192)
Mixed_5a has shape (?, 35, 35, 384)
Mixed_5b has shape (?, 35, 35, 384)
Mixed_5c has shape (?, 35, 35, 384)
Mixed_5d has shape (?, 35, 35, 384)
Mixed_5e has shape (?, 35, 35, 384)
Mixed_6a has shape (?, 17, 17, 1024)
Mixed_6b has shape (?, 17, 17, 1024)
Mixed_6c has shape (?, 17, 17, 1024)
Mixed_6d has shape (?, 17, 17, 1024)
Mixed_6e has shape (?, 17, 17, 1024)
Mixed_6f has shape (?, 17, 17, 1024)
Mixed_6g has shape (?, 17, 17, 1024)
Mixed_6h has shape (?, 17, 17, 1024)
Mixed_7a has shape (?, 8, 8, 1536)
Mixed_7b has shape (?, 8, 8, 1536)
Mixed_7c has shape (?, 8, 8, 1536)
Mixed_7d has shape (?, 8, 8, 1536)
AuxLogits has shape (?, 1001)
global_pool has shape (?, 1, 1, 1

# Load feature vector

In [7]:
# Load feature vector .h5 file 

features = h5py.File('./FV_inceptionV4.h5', 'r')
print("Loading feacture vectors...")

features_key = list(features.keys())
print("Available feature layer :", features_key)

# get feature vector at a layer
layer = 'AuxLogits'
print("==="*30)
print("Selected layer : ", layer)

features_value = features[layer][...]
a = np.array(features_value)
features_value = np.reshape(a, (a.shape[0], -1)) #depth가 1보다 클 때 depth 1의 벡터로 전환

features_filename = features['filenames'][...]
features_index = list(map(get_index, features_filename))

feat_cols = ['feature' + str(i) for i in range(features_value.shape[1])]
df = pd.DataFrame(features_value, columns = feat_cols)
df['index'] = features_index

print("# of images : ", df.shape[0], ", # of features : ", len(feat_cols))

min_value = np.amin(df[feat_cols].values)
max_value = np.amax(df[feat_cols].values)
print("min feature value:", min_value, ", max feature value: ", max_value)

#df[feat_cols] = df[feat_cols] - np.amin(df[feat_cols].values)
#print("min :", np.amin(df[feat_cols].values), "max : ", np.max(df[feat_cols].values))
#df[feat_cols] = df[feat_cols] / 10
#print("min :", np.amin(df[feat_cols].values), "max : ", np.max(df[feat_cols].values))

Loading feacture vectors...
Available feature layer : ['AuxLogits', 'Logits', 'PreLogitsFlatten', 'filenames', 'global_pool']
Selected layer :  AuxLogits
# of images :  697 , # of features :  1001
min feature value: -5.8775845 , max feature value:  11.935148


In [8]:
# check is any null value in dataframe
df[df.isnull().T.any().T]

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature992,feature993,feature994,feature995,feature996,feature997,feature998,feature999,feature1000,index
